In [37]:
import PIL
from PIL import Image
import torch
from typing import Union, List
import numpy as np
from text_gen import model, feature_extractor, tokenizer
import faiss
from image_retrieval import get_embed, pca_matrix


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
def init():
    classifier = torch.load('weights\\classifier_final2.pth', map_location=lambda storage, loc: storage)
    classifier.eval()
    generator = model
    generator = torch.load_state_dict('weights\\pytorch_model.bin', map_location=lambda storage, loc: storage)
    generator.eval()
    dim = image_vectors[0].shape[0]
    num_components = 64
    pca_matrix = faiss.PCAMatrix(dim, num_components)

    image_vectors = np.array(image_vectors)
    # обучаем PCAMatrix на исходных векторах
    pca_matrix.train(image_vectors)

    # применяем PCAMatrix для понижения размерности векторов
    output_vectors = pca_matrix.apply_py(image_vectors)

    index = faiss.IndexFlatL2(output_vectors[0].shape[0])

    index.add(output_vectors)
    index = faiss.read_index("weights\\working.index")


def find_simmilar_pca(embed_image):
    PCA = faiss.read_VectorTransform("weights\\PCA.pca")
    query_image_vector = pca_matrix.apply_py(np.array([embed_image]))

    k = 10
    distances, indices = index.search(query_image_vector, k)
    res = [df.iloc[i] for i in indices][0]
    return res



In [38]:
def predict(image: PIL.Image.Image, images: bool, group: bool, description: bool) -> dict:
    """
    images: похожие картинки
    group: класс экспоната
    description: описание экспоната
    

    returns
    {
        images: Union[pd.DataFrame, List[pd.Series]]
        
        group: str
        
        description: str
    }
    """
    response = {}
    vectors = get_embed(image)

    if images:
        response.update(images=find_simmilar_pca(vectors))
    if group:
        response.update(group=classifier(vectors))
    if description:
        response.update(description=generator(vectors))
    
    return response

def predict_caption(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds




In [39]:
relative_path = R"dataset\train\83137\20110231.jpg"
predict(Image.open('dataset/train/10008800/6885950.jpg'), True, True, True) # ['Самолет на взлетно-посадочной полосе аэропорта.']

AssertionError: 